In [13]:
import numpy as np
import pandas as pd 
import sklearn as sk
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [48]:
df=pd.read_csv("data/Telco_Customer_Churn.csv")
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [15]:
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [18]:
df['SeniorCitizen'].nunique()

2

In [49]:
X=df.drop(
        columns=['customerID','gender','PhoneService',
                'MultipleLines', 'PaperlessBilling','PaymentMethod', 'Churn'
        ], axis=1)
X[:5]

,SeniorCitizen,Partner,Dependents,tenure,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges
0,0,Yes,No,1,DSL,No,Yes,No,No,No,No,Month-to-month,29.85,29.85
1,0,No,No,34,DSL,Yes,No,Yes,No,No,No,One year,56.95,1889.5
2,0,No,No,2,DSL,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15
3,0,No,No,45,DSL,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75
4,0,No,No,2,Fiber optic,No,No,No,No,No,No,Month-to-month,70.70,151.65


In [28]:
df.index

RangeIndex(start=0, stop=7043, step=1)

In [51]:
df['SeniorCitizen']=df['SeniorCitizen'].astype(object)

In [50]:
y=df['Churn']

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=124)

numeric_features = ['tenure']
categorical_features = ['SeniorCitizen', 'Partner', 'Dependents', #'PhoneService',
                        'InternetService','OnlineSecurity','OnlineBackup',
                        'DeviceProtection','TechSupport','StreamingTV',
                        'StreamingMovies','Contract']

In [30]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression

without pipeline

In [31]:
imputer = SimpleImputer(strategy="median")
scaler = StandardScaler()
onehot = OneHotEncoder(handle_unknown="ignore", sparse=False)

In [32]:
imputer.fit(X_train[numeric_features])
imputed = imputer.transform(X_train[numeric_features])

scaler.fit(imputed)
scaled = scaler.transform(imputed)

In [34]:
onehot.fit(X_train[categorical_features])
onehotted = onehot.transform(X_train[categorical_features])

In [35]:
X_train_transformed = pd.DataFrame(np.concatenate((scaled, onehotted), axis=1))

In [36]:
model = LogisticRegression()
model.fit(X_train_transformed, y_train)

LogisticRegression()

In [39]:
X_test_filled = imputer.transform(X_test[numeric_features])
X_test_scaled = scaler.transform(X_test_filled)
X_test_onehot = onehot.transform(X_test[categorical_features])

In [41]:
type(X_test_scaled)
X_test_scaled[0]

array([1.55914459])

In [42]:
X_test_onehot[0]

array([1., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
       1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1.])

In [47]:
X_test = pd.DataFrame(np.concatenate((X_test_scaled, X_test_onehot), axis=1) )
model.score(X_test, y_test)
#y_pred = model.predict(X_test)
#accuracy(y_test, y_pred)

0.807382867960246

In [53]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

numeric_transformer = Pipeline(steps=[
    ('meanimputer', SimpleImputer(strategy="mean")),
    ("stdscaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown="ignore"))
])

col_transformer = ColumnTransformer(
    transformers=[('numeric_proc', numeric_transformer, numeric_features), 
                  ('cat_proc', categorical_transformer, categorical_features)
])

pipeline = Pipeline([
    ('col_tx', col_transformer),
    ('logit_regression', LogisticRegression())
])

pipeline.fit(X_train, y_train)
pipeline.score(X_test, y_test)

0.807382867960246

In [54]:
from sklearn import set_config
set_config(display='diagram')
pipeline

Pipeline(steps=[('col_tx',
                 ColumnTransformer(transformers=[('numeric_proc',
                                                  Pipeline(steps=[('meanimputer',
                                                                   SimpleImputer()),
                                                                  ('stdscaler',
                                                                   StandardScaler())]),
                                                  ['tenure']),
                                                 ('cat_proc',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['SeniorCitizen', 'Partner',
                                                   'Dependents',
                                                   'InternetService',
                                                   'OnlineSecurity',
                                                   'OnlineBackup',
                                                   'DeviceProtection',
                                                   'TechSupport', 'StreamingTV',
                                                   'StreamingMovies',
                                                   'Contract'])])),
                ('logit_regression', LogisticRegression())])

In [ ]:
pipeline.named_steps

In [ ]:
pipeline.named_steps['col_tx'].transformers_[0]

In [ ]:
pipeline.named_steps['col_tx'].transformers_[1]